In [36]:
import pandas as pd
import numpy as np

import dask.dataframe as dd
from sklearn.linear_model import LogisticRegression

from sklearn.pipeline import Pipeline, make_pipeline, FeatureUnion
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.impute import SimpleImputer

from sklearn.metrics import f1_score, classification_report
from sklearn.feature_selection import SelectFromModel

import pickle

In [48]:
def undersample_df_by_target(df, target_name):
    num_0 = len(df[df[target_name] == 0])
    num_1 = len(df[df[target_name] == 1])
    undersampled_data = pd.concat([df[df[target_name] == 0].sample(num_1), df[df[target_name] == 1]])

    return undersampled_data



In [49]:
def run_grid_search(estimator, X, y, params_grid, scoring='f1'):
    gsc = GridSearchCV(estimator, params_grid, scoring=scoring, cv=3, n_jobs=-1)

    gsc.fit(X, y)
    print("Best %s score: %.2f" % (scoring, gsc.best_score_))
    print()
    print("Best parameters set found on development set:")
    print()
    print(gsc.best_params_)
    print()
    print("Grid scores on development set:")
    print()

    for i, params in enumerate(gsc.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (gsc.cv_results_['mean_test_score'][i], gsc.cv_results_['std_test_score'][i] * 2, params))

    print()

    return gsc

In [50]:
def treshold_search(y_true, y_pred):
    top = [0.5, f1_score(y_true, y_pred[: , 1] > 0.5, average='macro')]
    for treshold in np.linspace(0, 1, 20):
        fscore = f1_score(y_true, y_pred[: , 1] > treshold, average='macro')
        if fscore > top[1]:
            top[0] = treshold
            top[1] = fscore
    print(f'Лучшая отсечка : {top[0]}, Метрика F1_macro: {top[1]}')
    print("=" * 80)
    print(classification_report(y_true, y_pred[:, 1] > top[0]))


In [51]:
def preprocess_data_train(prep_data_df, FEATURES_DATA):
    prep_data_df['buy_time'] = pd.to_datetime(prep_data_df['buy_time'], unit='s')
    #prep_data_df = prep_data_df.drop('Unnamed: 0', axis=1)
    prep_data_df['monthday'] = prep_data_df['buy_time'].dt.day
    prep_data_df = prep_data_df.sort_values('buy_time')
    prep_data_df['not_first_offer'] = prep_data_df.duplicated('id').astype(int)

    features_data_df = dd.read_csv(FEATURES_DATA, sep='\t')
    features_data_df = features_data_df.drop('Unnamed: 0', axis=1)
    train_list_index = list(prep_data_df['id'].unique())
    features_data_df = features_data_df.loc[features_data_df['id'].isin(train_list_index)].compute()
    features_data_df['buy_time'] = pd.to_datetime(features_data_df['buy_time'], unit='s')
    features_data_df = features_data_df.sort_values(by="buy_time")

    result_data = pd.merge_asof(prep_data_df, features_data_df, on='buy_time', by='id', direction='nearest')

    result_data.drop(['id', 'buy_time'], axis=1, inplace=True)
    result_data.drop_duplicates(inplace=True)
    
    result_data = result_data.set_index(['Unnamed: 0'])
    result_data.index.name = None
    result_data.sort_index(inplace=True)
    
    return result_data, train_list_index

In [52]:
def preprocess_data_test(prep_data_df, FEATURES_DATA, train_list_index):
    prep_data_df['buy_time'] = pd.to_datetime(prep_data_df['buy_time'], unit='s')
    #prep_data_df = prep_data_df.drop('Unnamed: 0', axis=1)
    prep_data_df['monthday'] = prep_data_df['buy_time'].dt.day
    prep_data_df = prep_data_df.sort_values('buy_time')
    prep_data_df['not_first_offer'] = (prep_data_df['id'].isin(train_list_index)).astype(int)

    features_data_df = dd.read_csv(FEATURES_DATA, sep='\t')
    features_data_df = features_data_df.drop('Unnamed: 0', axis=1)
    test_list_index = list(prep_data_df['id'].unique())
    features_data_df = features_data_df.loc[features_data_df['id'].isin(test_list_index)].compute()
    features_data_df['buy_time'] = pd.to_datetime(features_data_df['buy_time'], unit='s')
    features_data_df = features_data_df.sort_values(by="buy_time")

    result_data = pd.merge_asof(prep_data_df, features_data_df, on='buy_time', by='id', direction='nearest')

    result_data.drop(['id', 'buy_time'], axis=1, inplace=True)
    
    result_data = result_data.set_index(['Unnamed: 0'])
    result_data.index.name = None
    result_data.sort_index(inplace=True)
    
    return result_data

In [53]:
def select_type_cols(merged_data):
    X_nunique = merged_data.apply(lambda x: x.nunique(dropna=False))
    f_all = set(X_nunique.index.tolist())
    f_const = set(X_nunique[X_nunique == 1].index.tolist())
    f_categorical = set(X_nunique[X_nunique <= 30].index.tolist())
    f_numeric = (merged_data.fillna(0).astype(int).sum() - merged_data.fillna(0).sum()).abs()
    f_numeric = set(f_numeric[f_numeric > 0].index.tolist())
    f_binary = set(merged_data.loc[:, f_all].columns[(
            (merged_data.loc[:, f_all].max() == 1) & \
            (merged_data.loc[:, f_all].min() == 0) & \
            (merged_data.loc[:, f_all].isnull().sum() == 0))])
    f_categorical = f_categorical - f_const - f_binary
    f_numeric = f_numeric - f_categorical - f_const

    assert (X_nunique.shape[0] == len(f_const) + len(f_binary) + len(f_numeric) + len(f_categorical))

    f_all = list(f_binary | f_categorical | f_numeric)
    f_binary, f_categorical, f_numeric = list(f_binary), list(f_categorical), list(f_numeric)

    return f_all, f_binary, f_categorical, f_numeric


In [54]:
class ColumnSelector(BaseEstimator, TransformerMixin):
    def __init__(self, columns):
        self.columns = columns

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        assert isinstance(X, pd.DataFrame)

        try:
            return X[self.columns]
        except KeyError:
            cols_error = list(set(self.columns) - set(X.columns))
            raise KeyError("DataFrame не содердит следующие колонки: %s" % cols_error)

### Подготовка данных

In [55]:
train_df = pd.read_csv(TRAIN_DATA)

In [56]:
TRAIN_DATA = 'data/data_train.csv'
FEATURES_DATA = 'data/features.csv'
RANDOM_STATE = 9


In [57]:
train_df['buy_time'] = pd.to_datetime(train_df['buy_time'], unit='s')

In [58]:
X_train = undersample_df_by_target(train_df, 'target')

In [59]:
X_train, true_offers_ids = preprocess_data_train(X_train, FEATURES_DATA)

In [60]:
with open("data/offer_mark.txt", "w") as file:
    print(*true_offers_ids, file=file, sep="\n")

In [61]:
y_train = X_train['target']

In [62]:
X_train = X_train.drop('target', axis = 1)

In [63]:
f_all, f_binary, f_categorical, f_numeric = select_type_cols(X_train)

#### Обучение Модели

In [64]:
f_prep_pipeline = make_pipeline(
    ColumnSelector(columns=f_all),
    FeatureUnion(transformer_list=[
        ("numeric_features", make_pipeline(
            ColumnSelector(f_numeric),
            SimpleImputer(strategy="mean"),
            StandardScaler()
        )),
        ("categorical_features", make_pipeline(
            ColumnSelector(f_categorical),
            SimpleImputer(strategy="most_frequent"),
            OneHotEncoder(handle_unknown='ignore')
        )),
        ("boolean_features", make_pipeline(
            ColumnSelector(f_binary),
        ))
    ])
)

In [65]:
lg_fs_pipe = make_pipeline(
    f_prep_pipeline,
    SelectFromModel(LogisticRegression(penalty='l1', random_state=RANDOM_STATE, solver='liblinear'), max_features = 29),
    LogisticRegression(random_state=RANDOM_STATE)
)

In [66]:
lg_fs_pipe.fit(X_train, y_train)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Pipeline(steps=[('pipeline',
                 Pipeline(steps=[('columnselector',
                                  ColumnSelector(columns=['64', '80', '182',
                                                          '198', '166',
                                                          'monthday', '169',
                                                          '40', '184', '138',
                                                          '50', '5', '26',
                                                          '248', '127', '240',
                                                          '143', '171', '136',
                                                          '44', '74', '93',
                                                          '107', '188', '129',
                                                          '239', '157', '251',
                                                          '114', '124', ...])),
                                 ('featureunion',
                  

In [67]:
with open('data/lg_fs_model.pickle', 'wb') as f:
    pickle.dump(lg_fs_pipe, f, protocol=pickle.HIGHEST_PROTOCOL)

In [68]:
preds_proba_train = lg_fs_pipe.predict_proba(X_train)

In [69]:
treshold_search(y_train, preds_proba_train)

Лучшая отсечка : 0.5263157894736842, Метрика F1_macro: 0.8884699436162535
              precision    recall  f1-score   support

         0.0       0.91      0.87      0.89     60186
         1.0       0.87      0.91      0.89     60186

    accuracy                           0.89    120372
   macro avg       0.89      0.89      0.89    120372
weighted avg       0.89      0.89      0.89    120372

